# Importação de Bibliotecas

In [1]:
pip install pandera


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [49]:
pip install cassandra-driver

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import pandera as pa
import psycopg2
from cassandra.cluster import Cluster
from cassandra.connection import EndPoint

In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
from pyspark.sql.window import Window
from datetime import datetime, timedelta
from pandas import DataFrame

# Migração Postgres (nuvem) para Python (Pandas)


In [5]:

class connector_postgres:
    host = "34.151.204.5"
    database = "apps"
    user = "postgres"
    password = "secret"

    def conectar(self):
        conn = psycopg2.connect(host=self.host,
                                database=self.database,
                                user=self.user,
                                password=self.password)
        cursor = conn.cursor()

        return conn, cursor

    def desconectar(self, conn, cursor):
        cursor.close()
        conn.commit()
        conn.close()


    def selecionar(self, query):
        conn, cursor = self.conectar()
        cursor.execute(query)
        lista = []
        for result in cursor.fetchall():
            lista.append(result)
        self.desconectar(conn, cursor)
        return lista
              
        
conect = connector_postgres()


In [6]:


query1 = "SELECT * FROM entrada_apps_apps_treino_tratado"

apps_treino_tratado = pd.DataFrame(conect.selecionar(query1), columns=['instituicao', 'comentario', 'dataa', 'classificacao', 'elogio_quanto_ao_app', 'reclamacao_quanto_ao_app', 'elogio_a_instituicao', 'reclamacao_a_instituicao', 'nao_classificavel'])

query2 = "SELECT * FROM entrada_apps_apps_validacao_tratado"

apps_validacao_tratado = pd.DataFrame(conect.selecionar(query2), columns=['instituicao', 'comentario', 'dataa', 'classificacao', 'elogio_quanto_ao_app', 'reclamacao_quanto_ao_app', 'elogio_a_instituicao', 'reclamacao_a_instituicao', 'nao_classificavel'])

In [7]:
apps_treino_tratado

,instituicao,comentario,dataa,classificacao,elogio_quanto_ao_app,reclamacao_quanto_ao_app,elogio_a_instituicao,reclamacao_a_instituicao,nao_classificavel
0,Banco do Brasil,Excelente,28/07/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
1,Banco Itau,"muito bom, é pratico e facilita muito a vida e...",28/07/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
2,Banco Itau,Muito bom!,28/07/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
3,Banco Bradesco,"tenho utilizado para fazer pagamentos,transfer...",28/07/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
4,Nubank,o melhor cartão que existe.. sem falar da Nuco...,28/07/2019,5,NÃO,NÃO,NÃO,SIM,NÃO
...,...,...,...,...,...,...,...,...,...
8697,Caixa Econômica Federal,Muito bom,24/08/2020,5,SIM,NÃO,NÃO,NÃO,NÃO
8698,Caixa Econômica Federal,Top,24/08/2020,5,SIM,NÃO,NÃO,NÃO,NÃO
8699,Caixa Econômica Federal,A facilidade de resolver minhas movimentação b...,24/08/2020,5,SIM,NÃO,NÃO,NÃO,NÃO
8700,Caixa Econômica Federal,"Péssimo ,vou desinstalar",24/08/2020,1,NÃO,SIM,NÃO,NÃO,NÃO


In [8]:
apps_validacao_tratado

,instituicao,comentario,dataa,classificacao,elogio_quanto_ao_app,reclamacao_quanto_ao_app,elogio_a_instituicao,reclamacao_a_instituicao,nao_classificavel
0,Banco do Brasil,Muito bom o App. Sugiro uma melhoria na aba tr...,11/08/2019,4,SIM,SIM,NÃO,NÃO,NÃO
1,Banco do Brasil,excelente,11/08/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
2,Banco do Brasil,Muito eficiente e rápido,11/08/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
3,Banco do Brasil,muito útil no dia a dia!!!!!,11/08/2019,4,SIM,NÃO,NÃO,NÃO,NÃO
4,Banco do Brasil,"estou gostando muito, pois é bem prático e fác...",11/08/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
...,...,...,...,...,...,...,...,...,...
1370,Caixa Econômica Federal,Ótimo,24/08/2020,5,SIM,NÃO,NÃO,NÃO,NÃO
1371,Caixa Econômica Federal,Ótima,24/08/2020,5,SIM,NÃO,NÃO,NÃO,NÃO
1372,Caixa Econômica Federal,Ta melhorando,24/08/2020,4,SIM,NÃO,NÃO,NÃO,NÃO
1373,Caixa Econômica Federal,Deveria se mais fácil pra pagar contas,24/08/2020,2,NÃO,SIM,NÃO,NÃO,NÃO


# trocar de nome o df

In [9]:
df1 = apps_treino_tratado.copy()
df2 = apps_validacao_tratado.copy()

# Backup

In [10]:
backup_df1 = apps_treino_tratado.copy()
backup_df2 = apps_validacao_tratado.copy()

# Comparando nome das colunas

In [11]:
df1.dtypes


instituicao                 object
comentario                  object
dataa                       object
classificacao                int64
elogio_quanto_ao_app        object
reclamacao_quanto_ao_app    object
elogio_a_instituicao        object
reclamacao_a_instituicao    object
nao_classificavel           object
dtype: object

In [12]:
df2.dtypes

instituicao                 object
comentario                  object
dataa                       object
classificacao                int64
elogio_quanto_ao_app        object
reclamacao_quanto_ao_app    object
elogio_a_instituicao        object
reclamacao_a_instituicao    object
nao_classificavel           object
dtype: object

# Concatenação

In [13]:
df = pd.concat([df1,df2], axis=0, ignore_index=True)

In [14]:
df

,instituicao,comentario,dataa,classificacao,elogio_quanto_ao_app,reclamacao_quanto_ao_app,elogio_a_instituicao,reclamacao_a_instituicao,nao_classificavel
0,Banco do Brasil,Excelente,28/07/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
1,Banco Itau,"muito bom, é pratico e facilita muito a vida e...",28/07/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
2,Banco Itau,Muito bom!,28/07/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
3,Banco Bradesco,"tenho utilizado para fazer pagamentos,transfer...",28/07/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
4,Nubank,o melhor cartão que existe.. sem falar da Nuco...,28/07/2019,5,NÃO,NÃO,NÃO,SIM,NÃO
...,...,...,...,...,...,...,...,...,...
10072,Caixa Econômica Federal,Ótimo,24/08/2020,5,SIM,NÃO,NÃO,NÃO,NÃO
10073,Caixa Econômica Federal,Ótima,24/08/2020,5,SIM,NÃO,NÃO,NÃO,NÃO
10074,Caixa Econômica Federal,Ta melhorando,24/08/2020,4,SIM,NÃO,NÃO,NÃO,NÃO
10075,Caixa Econômica Federal,Deveria se mais fácil pra pagar contas,24/08/2020,2,NÃO,SIM,NÃO,NÃO,NÃO


# Remover linha duplicadas

In [15]:
df = df.drop_duplicates()

In [16]:
df

,instituicao,comentario,dataa,classificacao,elogio_quanto_ao_app,reclamacao_quanto_ao_app,elogio_a_instituicao,reclamacao_a_instituicao,nao_classificavel
0,Banco do Brasil,Excelente,28/07/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
1,Banco Itau,"muito bom, é pratico e facilita muito a vida e...",28/07/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
2,Banco Itau,Muito bom!,28/07/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
3,Banco Bradesco,"tenho utilizado para fazer pagamentos,transfer...",28/07/2019,5,SIM,NÃO,NÃO,NÃO,NÃO
4,Nubank,o melhor cartão que existe.. sem falar da Nuco...,28/07/2019,5,NÃO,NÃO,NÃO,SIM,NÃO
...,...,...,...,...,...,...,...,...,...
10071,Nubank,Ameiiii muito bom,24/08/2020,5,SIM,NÃO,NÃO,NÃO,NÃO
10072,Caixa Econômica Federal,Ótimo,24/08/2020,5,SIM,NÃO,NÃO,NÃO,NÃO
10074,Caixa Econômica Federal,Ta melhorando,24/08/2020,4,SIM,NÃO,NÃO,NÃO,NÃO
10075,Caixa Econômica Federal,Deveria se mais fácil pra pagar contas,24/08/2020,2,NÃO,SIM,NÃO,NÃO,NÃO


# Renomear nome da coluna

In [17]:
# Renomear coluna dataa para data

df.rename(columns={'dataa': 'data'}, inplace = True)

/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [18]:
df.dtypes

instituicao                 object
comentario                  object
data                        object
classificacao                int64
elogio_quanto_ao_app        object
reclamacao_quanto_ao_app    object
elogio_a_instituicao        object
reclamacao_a_instituicao    object
nao_classificavel           object
dtype: object

In [19]:
# Verificando inconsistências na coluna 'instituicao'

sorted(pd.unique(df['instituicao']))

['Banco Bradesco',
 'Banco Itau',
 'Banco do Brasil',
 'Caixa Econômica Federal',
 'Nubank']

In [20]:
# Verificando inconsistências na coluna 'classificacao'

sorted(pd.unique(df['classificacao']))

[1, 2, 3, 4, 5]

In [21]:
# Verificando inconsistências na coluna 'elogio_quanto_ao_app'

sorted(pd.unique(df['elogio_quanto_ao_app']))


['NAO', 'NÃO', 'Não', 'SIM']

In [22]:
# fazendo a alteração da palavra NAO
df.elogio_quanto_ao_app.replace('NÃO', 'NAO', inplace=True)
df.elogio_quanto_ao_app.replace('Não', 'NAO', inplace=True)

/opt/conda/miniconda3/lib/python3.8/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [23]:
#confirmando alteração
sorted(pd.unique(df['elogio_quanto_ao_app']))

['NAO', 'SIM']

In [24]:
# Verificando inconsistências na coluna 'reclamacao_quanto_ao_app'

sorted(pd.unique(df['reclamacao_quanto_ao_app']))

['NÃO', 'Não', 'SIM', 'nÃO']

In [25]:
# fazendo a alteração da palavra NAO
df.reclamacao_quanto_ao_app.replace('NÃO', 'NAO', inplace=True)
df.reclamacao_quanto_ao_app.replace('Não', 'NAO', inplace=True)
df.reclamacao_quanto_ao_app.replace('nÃO', 'NAO', inplace=True)

In [26]:
# Verificando inconsistências na coluna 'reclamacao_quanto_ao_app'

sorted(pd.unique(df['reclamacao_quanto_ao_app']))

['NAO', 'SIM']

In [27]:
# Verificando inconsistências na coluna 'elogio_a_instituicao'

sorted(pd.unique(df['elogio_a_instituicao']))

['NÃO', 'Não', 'SIM']

In [28]:
# fazendo a alteração da palavra NAO
df.elogio_a_instituicao.replace('NÃO', 'NAO', inplace=True)
df.elogio_a_instituicao.replace('Não', 'NAO', inplace=True)

In [29]:
# Verificando inconsistências na coluna 'elogio_a_instituicao'

sorted(pd.unique(df['elogio_a_instituicao']))

['NAO', 'SIM']

In [30]:
# Verificando inconsistências na coluna 'reclamacao_a_instituicao'

sorted(pd.unique(df['reclamacao_a_instituicao']))

['NÃO', 'Não', 'SIM', 'Sim']

In [31]:
# fazendo a alteração da palavra NAO
df.reclamacao_a_instituicao.replace('NÃO', 'NAO', inplace=True)
df.reclamacao_a_instituicao.replace('Não', 'NAO', inplace=True)
df.reclamacao_a_instituicao.replace('Sim', 'SIM', inplace=True)

In [32]:
# Verificando inconsistências na coluna 'reclamacao_a_instituicao'

sorted(pd.unique(df['reclamacao_a_instituicao']))

['NAO', 'SIM']

In [33]:
# Verificando inconsistências na coluna 'nao_classificavel'

sorted(pd.unique(df['nao_classificavel']))

['NÃO', 'Não', 'SIM']

In [34]:
# fazendo a alteração da palavra NAO
df.nao_classificavel.replace('NÃO', 'NAO', inplace=True)
df.nao_classificavel.replace('Não', 'NAO', inplace=True)

In [35]:
# Verificando inconsistências na coluna 'nao_classificavel'

sorted(pd.unique(df['nao_classificavel']))

['NAO', 'SIM']

In [68]:
# convertendo a coluna de objeto para data
#df['data'] = pd.to_datetime(df['data'], dayfirst = True)
#df = df.astype({'data':'str'})

In [69]:
df.dtypes

instituicao                 object
comentario                  object
data                        object
classificacao                int64
elogio_quanto_ao_app        object
reclamacao_quanto_ao_app    object
elogio_a_instituicao        object
reclamacao_a_instituicao    object
nao_classificavel           object
dtype: object

In [70]:
df


,instituicao,comentario,data,classificacao,elogio_quanto_ao_app,reclamacao_quanto_ao_app,elogio_a_instituicao,reclamacao_a_instituicao,nao_classificavel
0,Banco do Brasil,Excelente,2019-07-28,5,SIM,NAO,NAO,NAO,NAO
1,Banco Itau,"muito bom, é pratico e facilita muito a vida e...",2019-07-28,5,SIM,NAO,NAO,NAO,NAO
2,Banco Itau,Muito bom!,2019-07-28,5,SIM,NAO,NAO,NAO,NAO
3,Banco Bradesco,"tenho utilizado para fazer pagamentos,transfer...",2019-07-28,5,SIM,NAO,NAO,NAO,NAO
4,Nubank,o melhor cartão que existe.. sem falar da Nuco...,2019-07-28,5,NAO,NAO,NAO,SIM,NAO
...,...,...,...,...,...,...,...,...,...
10071,Nubank,Ameiiii muito bom,2020-08-24,5,SIM,NAO,NAO,NAO,NAO
10072,Caixa Econômica Federal,Ótimo,2020-08-24,5,SIM,NAO,NAO,NAO,NAO
10074,Caixa Econômica Federal,Ta melhorando,2020-08-24,4,SIM,NAO,NAO,NAO,NAO
10075,Caixa Econômica Federal,Deveria se mais fácil pra pagar contas,2020-08-24,2,NAO,SIM,NAO,NAO,NAO


# Pandera

In [72]:
df.dtypes

instituicao                 object
comentario                  object
data                        object
classificacao                int64
elogio_quanto_ao_app        object
reclamacao_quanto_ao_app    object
elogio_a_instituicao        object
reclamacao_a_instituicao    object
nao_classificavel           object
dtype: object

In [71]:
schema = pa.DataFrameSchema(
    columns = {
        "instituicao":pa.Column(pa.String,pa.Check.str_length(0,150)),
        "comentario":pa.Column(pa.String,pa.Check.str_length(0,2000), nullable=True),
        "data":pa.Column(pa.String,pa.Check.str_length(0,10)),
        "classificacao":pa.Column(pa.Int),
        "elogio_quanto_ao_app":pa.Column(pa.String,pa.Check.str_length(0,10), nullable=True),
        "reclamacao_quanto_ao_app":pa.Column(pa.String,pa.Check.str_length(0,10), nullable=True),
        "elogio_a_instituicao":pa.Column(pa.String,pa.Check.str_length(0,10), nullable=True),
        "reclamacao_a_instituicao":pa.Column(pa.String,pa.Check.str_length(0,10), nullable=True),
        "nao_classificavel":pa.Column(pa.String,pa.Check.str_length(0,150), nullable=True)
    }
)    

In [73]:
schema.validate(df)

,instituicao,comentario,data,classificacao,elogio_quanto_ao_app,reclamacao_quanto_ao_app,elogio_a_instituicao,reclamacao_a_instituicao,nao_classificavel
0,Banco do Brasil,Excelente,2019-07-28,5,SIM,NAO,NAO,NAO,NAO
1,Banco Itau,"muito bom, é pratico e facilita muito a vida e...",2019-07-28,5,SIM,NAO,NAO,NAO,NAO
2,Banco Itau,Muito bom!,2019-07-28,5,SIM,NAO,NAO,NAO,NAO
3,Banco Bradesco,"tenho utilizado para fazer pagamentos,transfer...",2019-07-28,5,SIM,NAO,NAO,NAO,NAO
4,Nubank,o melhor cartão que existe.. sem falar da Nuco...,2019-07-28,5,NAO,NAO,NAO,SIM,NAO
...,...,...,...,...,...,...,...,...,...
10071,Nubank,Ameiiii muito bom,2020-08-24,5,SIM,NAO,NAO,NAO,NAO
10072,Caixa Econômica Federal,Ótimo,2020-08-24,5,SIM,NAO,NAO,NAO,NAO
10074,Caixa Econômica Federal,Ta melhorando,2020-08-24,4,SIM,NAO,NAO,NAO,NAO
10075,Caixa Econômica Federal,Deveria se mais fácil pra pagar contas,2020-08-24,2,NAO,SIM,NAO,NAO,NAO


In [74]:
apps_pandas = df.copy()

# Salvar

In [75]:
#df.to_parquet('gs://deloitte_g10/saída/Pandas/Apps/apps_normalizado.parquet')
apps_pandas.to_parquet('gs://deloitte_g10/saída/Pandas/Apps/apps_normalizado.snappy', compression='snappy')


# Backup Final

In [76]:
backup_apps_pandas = apps_pandas.copy()

# MIgração para Cassandra

In [77]:
# Passando dados para o cassandra

cluster = Cluster (['35.198.46.127'])
session = cluster.connect('apps')

In [78]:
# Salvar em uma lista 
lista = df.values.tolist()


In [79]:
len(lista)

9366

In [80]:
# Passando os valores para o Cassandra

for i in lista:
    query = "INSERT INTO apps_tratado (chave, instituicao, comentario, data, classificacao, elogio_quanto_ao_app, reclamacao_quanto_ao_app, elogio_a_instituicao, reclamacao_a_instituicao, nao_classificavel ) VALUES(uuid(),'{}','{}','{}',{},'{}','{}','{}','{}','{}')".format(i[0], i[1], i[2], i[3], i[4], i[5], i[6], i[7], i[8])
    session.execute(query)

In [81]:
# Desligando a conexão com o Cassandra

cluster.shutdown()

In [82]:
print(lista[0])


['Banco do Brasil', 'Excelente', '2019-07-28', 5, 'SIM', 'NAO', 'NAO', 'NAO', 'NAO']


# MIgração de Cassandra para Pyspark

## Importando dados do Cassandra para uma lista

In [86]:
# Conectando com a VM do Cassandra
cluster = Cluster(['35.198.46.127'])
                               
session = cluster.connect('apps')

In [87]:
# Importar os dados do Cassandra para uma lista

query1 = "SELECT * FROM apps_tratado"

saida = session.execute(query1)
dados = []
for i in saida:
    dados.append(i)

In [88]:
# Imprimir a lista com os dados ver tamanho da lista 

len(dados)

9366

In [89]:
# Cortando a conexão com o Cassandra

cluster.shutdown()

In [90]:
# Salvar os dados em um DataFrame de Pandas

df_dados = pd.DataFrame(dados)

In [92]:
df_dados.dtypes

chave                       object
classificacao                int64
comentario                  object
data                        object
elogio_a_instituicao        object
elogio_quanto_ao_app        object
instituicao                 object
nao_classificavel           object
reclamacao_a_instituicao    object
reclamacao_quanto_ao_app    object
dtype: object

In [94]:
# Excluir coluna de chave

df_dados.drop(columns=['chave'], inplace = True)

In [95]:
df_dados.dtypes

classificacao                int64
comentario                  object
data                        object
elogio_a_instituicao        object
elogio_quanto_ao_app        object
instituicao                 object
nao_classificavel           object
reclamacao_a_instituicao    object
reclamacao_quanto_ao_app    object
dtype: object

In [96]:
# Criando a SparkSession

spark = (
    SparkSession
      .builder
      .master('local')
      .appName('apps')
      .config('spark.ui.port', '4050')
      .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [97]:
apps_spark = spark .createDataFrame(df_dados)

In [98]:
apps_spark.show()

22/01/25 12:48:03 WARN org.apache.spark.scheduler.TaskSetManager: Stage 0 contains a task of very large size (1158 KiB). The maximum recommended task size is 1000 KiB.


+-------------+--------------------+----------+--------------------+--------------------+--------------------+-----------------+------------------------+------------------------+
|classificacao|          comentario|      data|elogio_a_instituicao|elogio_quanto_ao_app|         instituicao|nao_classificavel|reclamacao_a_instituicao|reclamacao_quanto_ao_app|
+-------------+--------------------+----------+--------------------+--------------------+--------------------+-----------------+------------------------+------------------------+
|            5|        Gostei muito|2019-12-11|                 NAO|                 SIM|              Nubank|              NAO|                     NAO|                     NAO|
|            3|O app é excelente...|2019-09-08|                 NAO|                 NAO|          Banco Itau|              NAO|                     NAO|                     SIM|
|            1|Péssimo. Fiz uma ...|2020-08-09|                 NAO|                 NAO|              Nu

In [102]:
apps_spark = apps_spark.withColumn('data', apps_spark['data'].cast(DateType()))

In [103]:
apps_spark.printSchema()

root
 |-- classificacao: long (nullable = true)
 |-- comentario: string (nullable = true)
 |-- data: date (nullable = true)
 |-- elogio_a_instituicao: string (nullable = true)
 |-- elogio_quanto_ao_app: string (nullable = true)
 |-- instituicao: string (nullable = true)
 |-- nao_classificavel: string (nullable = true)
 |-- reclamacao_a_instituicao: string (nullable = true)
 |-- reclamacao_quanto_ao_app: string (nullable = true)



In [105]:
# Renomeando colunas 1

apps_spark = (apps_spark.withColumnRenamed('elogio_quanto_ao_app', 'elogio_app')
      .withColumnRenamed('reclamacao_quanto_ao_app', 'reclamacao_app')
      .withColumnRenamed('elogio_a_instituicao', 'elogio_instituicao')
      .withColumnRenamed('reclamacao_a_instituicao', 'reclamacao_instituicao')
      
)

In [106]:
apps_spark.printSchema()

root
 |-- classificacao: long (nullable = true)
 |-- comentario: string (nullable = true)
 |-- data: date (nullable = true)
 |-- elogio_instituicao: string (nullable = true)
 |-- elogio_app: string (nullable = true)
 |-- instituicao: string (nullable = true)
 |-- nao_classificavel: string (nullable = true)
 |-- reclamacao_instituicao: string (nullable = true)
 |-- reclamacao_app: string (nullable = true)



In [108]:
# Reordenando colunas

ordem = ['data', 'instituicao', 'classificacao', 'elogio_app', 'reclamacao_app', 'elogio_instituicao', 'reclamacao_instituicao', 'comentario']
apps_spark = apps_spark.select(ordem)

In [109]:
apps_spark.printSchema()

root
 |-- data: date (nullable = true)
 |-- instituicao: string (nullable = true)
 |-- classificacao: long (nullable = true)
 |-- elogio_app: string (nullable = true)
 |-- reclamacao_app: string (nullable = true)
 |-- elogio_instituicao: string (nullable = true)
 |-- reclamacao_instituicao: string (nullable = true)
 |-- comentario: string (nullable = true)



In [111]:
# Ordenar linhas por data

apps_spark = apps_spark.orderBy('data')

In [115]:
apps_spark.show(10)

+----------+--------------------+-------------+----------+--------------+------------------+----------------------+--------------------+
|      data|         instituicao|classificacao|elogio_app|reclamacao_app|elogio_instituicao|reclamacao_instituicao|          comentario|
+----------+--------------------+-------------+----------+--------------+------------------+----------------------+--------------------+
|2019-07-21|      Banco Bradesco|            4|       NAO|           SIM|               NAO|                   NAO|estou tentando fa...|
|2019-07-21|     Banco do Brasil|            4|       SIM|           NAO|               NAO|                   NAO|Muito bom e fácil...|
|2019-07-21|     Banco do Brasil|            5|       SIM|           NAO|               NAO|                   NAO|               otimo|
|2019-07-21|              Nubank|            1|       NAO|           SIM|               NAO|                   NAO|Affffff o que est...|
|2019-07-21|              Nubank|        

22/01/25 13:05:27 WARN org.apache.spark.scheduler.TaskSetManager: Stage 4 contains a task of very large size (1158 KiB). The maximum recommended task size is 1000 KiB.


In [133]:
# instituição separado por classificação 
apps_spark.select('instituicao', 'classificacao').groupBy('instituicao', 'classificacao') \
                                              .count() \
                                              .sort("instituicao", "classificacao", ascending=False).show(100)

+--------------------+-------------+-----+
|         instituicao|classificacao|count|
+--------------------+-------------+-----+
|              Nubank|            5| 1279|
|              Nubank|            4|  153|
|              Nubank|            3|   87|
|              Nubank|            2|   89|
|              Nubank|            1|  289|
|Caixa Econômica F...|            5| 1044|
|Caixa Econômica F...|            4|  199|
|Caixa Econômica F...|            3|  121|
|Caixa Econômica F...|            2|  115|
|Caixa Econômica F...|            1|  395|
|     Banco do Brasil|            5| 1380|
|     Banco do Brasil|            4|  208|
|     Banco do Brasil|            3|   75|
|     Banco do Brasil|            2|   42|
|     Banco do Brasil|            1|  132|
|          Banco Itau|            5| 1217|
|          Banco Itau|            4|  192|
|          Banco Itau|            3|  102|
|          Banco Itau|            2|   89|
|          Banco Itau|            1|  311|
|      Banc

22/01/25 18:30:48 WARN org.apache.spark.scheduler.TaskSetManager: Stage 29 contains a task of very large size (1158 KiB). The maximum recommended task size is 1000 KiB.


In [125]:
# número de avaliações por instituição
apps_spark.select('instituicao').groupBy('instituicao') \
                                              .count() \
                                              .sort("instituicao", ascending=False).show(100)

22/01/25 13:59:47 WARN org.apache.spark.scheduler.TaskSetManager: Stage 26 contains a task of very large size (1158 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+
|         instituicao|count|
+--------------------+-----+
|              Nubank| 1897|
|Caixa Econômica F...| 1874|
|     Banco do Brasil| 1837|
|          Banco Itau| 1911|
|      Banco Bradesco| 1847|
+--------------------+-----+



In [124]:
# Verificar quantas pessoas elogiaram a instituição em relação ao app ordenando por nome 
apps_spark.select('instituicao', 'elogio_app').groupBy('instituicao', 'elogio_app') \
                                              .count() \
                                              .sort("count", ascending=True).show()


+--------------------+----------+-----+
|         instituicao|elogio_app|count|
+--------------------+----------+-----+
|     Banco do Brasil|       NAO|  309|
|      Banco Bradesco|       NAO|  433|
|          Banco Itau|       NAO|  568|
|Caixa Econômica F...|       NAO|  731|
|              Nubank|       NAO|  747|
|Caixa Econômica F...|       SIM| 1143|
|              Nubank|       SIM| 1150|
|          Banco Itau|       SIM| 1343|
|      Banco Bradesco|       SIM| 1414|
|     Banco do Brasil|       SIM| 1528|
+--------------------+----------+-----+



22/01/25 13:55:36 WARN org.apache.spark.scheduler.TaskSetManager: Stage 23 contains a task of very large size (1158 KiB). The maximum recommended task size is 1000 KiB.


In [120]:
# Verificar quantas pessoas reclaram da instituação em relação ao app sim ou não
apps_spark.select('instituicao', 'reclamacao_app').groupBy('instituicao', 'reclamacao_app') \
                                              .count() \
                                              .sort("instituicao", "reclamacao_app", ascending=True).show()


22/01/25 13:44:41 WARN org.apache.spark.scheduler.TaskSetManager: Stage 11 contains a task of very large size (1158 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------+-----+
|         instituicao|reclamacao_app|count|
+--------------------+--------------+-----+
|      Banco Bradesco|           NAO| 1459|
|      Banco Bradesco|           SIM|  388|
|          Banco Itau|           NAO| 1376|
|          Banco Itau|           SIM|  535|
|     Banco do Brasil|           NAO| 1570|
|     Banco do Brasil|           SIM|  267|
|Caixa Econômica F...|           NAO| 1197|
|Caixa Econômica F...|           SIM|  677|
|              Nubank|           NAO| 1542|
|              Nubank|           SIM|  355|
+--------------------+--------------+-----+



In [121]:
# Verificar a avaliação das pessoas referente a instituição, se elogiam  sim ou não
apps_spark.select('instituicao', 'elogio_instituicao').groupBy('instituicao', 'elogio_instituicao') \
                                              .count() \
                                              .sort("instituicao", "elogio_instituicao", ascending=True).show()


22/01/25 13:44:44 WARN org.apache.spark.scheduler.TaskSetManager: Stage 14 contains a task of very large size (1158 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+------------------+-----+
|         instituicao|elogio_instituicao|count|
+--------------------+------------------+-----+
|      Banco Bradesco|               NAO| 1808|
|      Banco Bradesco|               SIM|   39|
|          Banco Itau|               NAO| 1855|
|          Banco Itau|               SIM|   56|
|     Banco do Brasil|               NAO| 1793|
|     Banco do Brasil|               SIM|   44|
|Caixa Econômica F...|               NAO| 1859|
|Caixa Econômica F...|               SIM|   15|
|              Nubank|               NAO| 1588|
|              Nubank|               SIM|  309|
+--------------------+------------------+-----+



In [122]:
# Verificar a avaliação das pessoas referente a instituição, se tem alguma reclamação sim ou não
apps_spark.select('instituicao', 'reclamacao_instituicao').groupBy('instituicao', 'reclamacao_instituicao') \
                                              .count() \
                                              .sort("instituicao", "reclamacao_instituicao", ascending=True).show()


+--------------------+----------------------+-----+
|         instituicao|reclamacao_instituicao|count|
+--------------------+----------------------+-----+
|      Banco Bradesco|                   NAO| 1799|
|      Banco Bradesco|                   SIM|   48|
|          Banco Itau|                   NAO| 1836|
|          Banco Itau|                   SIM|   75|
|     Banco do Brasil|                   NAO| 1788|
|     Banco do Brasil|                   SIM|   49|
|Caixa Econômica F...|                   NAO| 1830|
|Caixa Econômica F...|                   SIM|   44|
|              Nubank|                   NAO| 1597|
|              Nubank|                   SIM|  300|
+--------------------+----------------------+-----+



22/01/25 13:44:48 WARN org.apache.spark.scheduler.TaskSetManager: Stage 17 contains a task of very large size (1158 KiB). The maximum recommended task size is 1000 KiB.


In [128]:
# Criando a tabela temporária para consultas SQL

apps_spark.createOrReplaceTempView('apps')

In [130]:
# Conferindo tipos de dados da tabela temporária

spark.sql('''
DESCRIBE apps

''').show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|                data|     date|   null|
|         instituicao|   string|   null|
|       classificacao|   bigint|   null|
|          elogio_app|   string|   null|
|      reclamacao_app|   string|   null|
|  elogio_instituicao|   string|   null|
|reclamacao_instit...|   string|   null|
|          comentario|   string|   null|
+--------------------+---------+-------+



In [132]:
backup_apps_spark = apps_spark.alias('backup_apps_spark')

In [134]:
(apps_spark.write.format("parquet")
.option("header", "true")
.option("inferschema", "true")
.save("gs://deloitte_g10/saída/Pyspark/Apps/apps_spark_norm")
)

22/01/25 18:37:16 WARN org.apache.spark.scheduler.TaskSetManager: Stage 32 contains a task of very large size (1158 KiB). The maximum recommended task size is 1000 KiB.
22/01/25 18:37:16 WARN org.apache.spark.scheduler.TaskSetManager: Stage 33 contains a task of very large size (1158 KiB). The maximum recommended task size is 1000 KiB.
